# Generate tiles for high resolution global extent map

[Data sources](https://docs.google.com/document/d/1D9kGrF9AUXDrMLqm-51ZfI43vlfYqpzcQd3mk7MIQqk/edit#heading=h.z8hx51w9ls2m)


In [1]:
import os
from pathlib import Path
import geopandas
import subprocess
import logging
import requests
import gzip
import fiona
from typing import Union
import pathlib
import dotenv

%run ../utils_mapbox.ipynb


In [2]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets'
#BASE_DIR = f'{WORK_DIR.parents[3]}/data'
logging.basicConfig(level=logging.INFO)

# @TODO: Add expected data files source as an environment variable. and add the download bit for the data sources.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'
outFolder= Path(f'{BASE_DIR}/hi_res_extent')
#outputPath = f'{outFolder.parents[1]}/processed'

In [3]:
!ls {outFolder}

gmw_v4_sen2_sgl_cls_v007.gpkg           gmw_v4_sen2_sgl_cls_v007_wgs84.geojson
gmw_v4_sen2_sgl_cls_v007.gpkg.gz        gmw_v4_sen2_sgl_cls_v007_z2_z12.mbtiles
gmw_v4_sen2_sgl_cls_v007_ZG.mbtiles


In [13]:
df_extent = geopandas.read_file(f'{outFolder}/gmw_v4_sen2_sgl_cls_v007.gpkg')
df_extent.head(2)

,ClassID,geometry
0,1,"POLYGON ((35.66950 22.94630, 35.66960 22.94630..."
1,1,"POLYGON ((35.66620 22.94670, 35.66620 22.94660..."


In [14]:
df_extent.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [15]:
#df_extent.to_crs(epsg=4326, inplace=True)

In [16]:
df_extent.to_file(f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson', driver='GeoJSON')

In [17]:
!ls -la {outFolder}

total 15408840
drwxr-xr-x@   8 angel  staff         256 Jul 18 15:14 .
drwxr-xr-x@ 123 angel  staff        3936 Jul 18 13:22 ..
-rw-r--r--@   1 angel  staff        6148 Jan 16  2024 .DS_Store
-rw-r--r--@   1 angel  staff  2756218880 Jul 18 13:24 gmw_v4_sen2_sgl_cls_v007.gpkg
-rw-r--r--@   1 angel  staff   520471287 Jul 18 12:14 gmw_v4_sen2_sgl_cls_v007.gpkg.gz
-rw-r--r--@   1 angel  staff   388587520 Jun 26  2023 gmw_v4_sen2_sgl_cls_v007_ZG.mbtiles
-rw-r--r--@   1 angel  staff  3451417206 Jul 18 15:33 gmw_v4_sen2_sgl_cls_v007_wgs84.geojson
-rw-r--r--@   1 angel  staff   756154368 Jun 26  2023 gmw_v4_sen2_sgl_cls_v007_wgs84.mbtiles


In [18]:
data_path = f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson'
output_path = f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_z2_z12.mbtiles'

!tippecanoe -Z2 -z12 -f -P -o {output_path} --extend-zooms-if-still-dropping {data_path}
print("Done!")

For layer 0, using name "gmw_v4_sen2_sgl_cls_v007_wgs84"
/Users/angel/Documents/REPOSITORIOS/mangrove-atlas/data/data/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson:474324: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","crs":{"type":"name","properties":{"name":"urn:ogc:def:crs:OGC:1.3:CRS84"}},"features":[]}
/Users/angel/Documents/REPOSITORIOS/mangrove-atlas/data/data/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_wgs84.geojson:389349: Found ] at top level
3342900 features, 459456118 bytes of geometry, 836 bytes of separate metadata, 96 bytes of string pool
  99.9%  12/3385/2013  
Done!


In [8]:
#get mapbox user and token from .env file located in ../../../../.env
dotenv.load_dotenv(f'{WORK_DIR.parents[3]}/.env')
MAPBOX_USER = os.getenv('MAPBOX_USER')
MAPBOX_TOKEN = os.getenv('MAPBOX_VECTOR_TOKEN')

output_path = f'{outFolder}/gmw_v4_sen2_sgl_cls_v007_z2_z12.mbtiles'

In [12]:
uploadToMapbox(Path(output_path), 'gmw_v4_sen2_sgl_cls_v007_wgs84', MAPBOX_USER, MAPBOX_TOKEN)

INFO:__main__:Uploading to Mapbox...
INFO:__main__:Uploading to S3...


upload: ../../../../data/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_z2_z12.mbtiles to s3://tilestream-tilesets-production/0d/_pending/sxhna22urmm1h5u0i23ifrylc/globalmangrovewatch


INFO:__main__:CompletedProcess(args='aws s3 cp /Users/angel/Documents/REPOSITORIOS/mangrove-atlas/data/data/hi_res_extent/gmw_v4_sen2_sgl_cls_v007_z2_z12.mbtiles s3://tilestream-tilesets-production/0d/_pending/sxhna22urmm1h5u0i23ifrylc/globalmangrovewatch --region us-east-1', returncode=0)


ConnectionError: HTTPSConnectionPool(host='api.mapbox.com', port=443): Max retries exceeded with url: /uploads/v1/globalmangrovewatch?access_token=sk.eyJ1IjoiZ2xvYmFsbWFuZ3JvdmV3YXRjaCIsImEiOiJjbHlyY280ZHMwNTR3MmpzbTA5ZG1lZDYzIn0.lBpNFLzlyTefkZ8QiCiNnQ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x16a728410>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))